In [1]:
import numpy as np

## TESTERS  ##############################
I = np.identity(5)
B = np.array([[1,1],
              [0,1]])
U = np.array([[0, 0, 0, 0, 0],
              [1, 0, 0, 0, 0],
              [1, 1, 0, 0, 0],
              [1, 0, 1, 0, 0],
              [0, 1, 0, 1, 0]])
P = np.array([[0, 0, 0, 0, 1],
              [1, 0, 0, 0, 1],
              [1, 1, 0, 0, 1],
              [1, 0, 1, 0, 1],
              [0, 1, 0, 1, 1]])
###########################################

## make_transition is a function that takes in an adjacency matrix and
## produces a transition matrix under a given sets of rules and using a
## probability damping factor in accordance with the pagerank system
def make_transition(A, p):
    (n, m) = A.shape
    big_c = np.zeros((n,n))

    # check if square
    if n == m:
        # get number of ones in a column
        for i in range(n):
            col = A[:, i]
            c = 0

            #count number of ones in a column
            for j in range(n):
                if col[j] == 1:
                    c = c + 1

            # a fix for if the column is of zeros, replaces with one
            if c == 0:
                col = np.ones(n)
                c = n

            # populate the big_C matrix with the correct values given
            # part 1
            for k in range(n):
                if col[k] == 1:
                    big_c[k][i] = (n + (c - n) * p)/(c * n)
                else:
                    big_c[k][i] = p/n

    # return the final array
    return big_c

a = make_transition(U, .8)
b = make_transition(P, .8)
c = make_transition(U, .4)



In [73]:
print("Transition Matrix:")
print(P, "\n")

print("Adjacency Matrix:")
print(make_transition(P, .8))

Transition Matrix:
[[0 0 0 0 1]
 [1 0 0 0 1]
 [1 1 0 0 1]
 [1 0 1 0 1]
 [0 1 0 1 1]] 

Adjacency Matrix:
[[0.16000 0.16000 0.16000 0.16000 0.20000]
 [0.22667 0.16000 0.16000 0.16000 0.20000]
 [0.22667 0.26000 0.16000 0.16000 0.20000]
 [0.22667 0.16000 0.36000 0.16000 0.20000]
 [0.16000 0.26000 0.16000 0.36000 0.20000]]


## Problem 3
We now desire to have some knowledge about the eigenvectors of our above
matrices. We start by formulating a bit of functionality that will become useful
soon, and then we start figuring out eigenvectors associated with the eigen
value 1.

In [2]:
e_val_b, e_vec_b = np.linalg.eig(b)
e_val_c, e_vec_c = np.linalg.eig(c)

In [3]:
float_formatter = "{:.4f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

def sum_vec(vec):
    sum = 0
    for i in range(len(vec)):
        sum = vec[i] + sum
    return sum

def normalize(vec):
    sum = np.sum(vec)
    norm_vec = vec / sum

    return norm_vec

def print_norm(vec):
    print(sum_vec(normalize(vec)))

In [75]:
## determine which eigenvalue is of value one
print("eigen val for first entry in e_val_b:  ", e_val_b[0])
print("eigen val for first entry in e_val_c:  ", e_val_c[0])


eigen val for first entry in e_val_b:   (1+0j)
eigen val for first entry in e_val_c:   (1.0000000000000004+0j)


Clearly the first entry (index zero) has the value one, so the associated
eigenvector in our matrix should be the associated 1-eigenvector

In [76]:
## name the vector of interest as the one eigenvector
ovb = e_vec_b[:,0]
print("ovb:  ", ovb)

ovc = e_vec_c[:,0]
print("ovc:  ", ovc)

ovb:   [-0.3759169 +0.j -0.40097802+0.j -0.44107583+0.j -0.48919319+0.j
 -0.51385334+0.j]
ovc:   [0.24682597+0.j 0.29619116+0.j 0.38504851+0.j 0.52722026+0.j
 0.65201547+0.j]


We would now like to normalize this eigenvector, as it needs to be a probability
vector, so it should sum to one...<br>

We do this by creating a new vector that is normalized. To do this, we divide
each entry in the vector by the sum of all the entries in the vector.

In [6]:
def norm_inf(ov):
    ## normalize ov
    ov_n = normalize(ov)
    print("NORMALISED VECTOR:\n", ov_n, "\n")

    ## Normalization test
    print("NORMALISATION TEST:")
    print(np.sum(ov_n))

    print()

    print("FORMATTED VECTOR:")
    for i in range(len(ov_n)):
        print(ov_n[i])

    print()
    print()

norm_inf(ovb)

NORMALISED VECTOR:
 [0.16925438-0.j 0.180538  -0.j 0.1985918 -0.j 0.22025636-0.j
 0.23135945-0.j] 

NORMALISATION TEST:
(1+0j)

FORMATTED VECTOR:
(0.1692543780465788-0j)
(0.18053800324968394-0j)
(0.19859180357465242-0j)
(0.22025636396461457-0j)
(0.23135945116447024-0j)




In [7]:
norm_inf(ovc)

NORMALISED VECTOR:
 [0.11712894+0.j 0.14055472+0.j 0.18272114+0.j 0.25018741+0.j
 0.3094078 +0.j] 

NORMALISATION TEST:
(1+0j)

FORMATTED VECTOR:
(0.11712893553223379+0j)
(0.14055472263868063+0j)
(0.1827211394302849+0j)
(0.25018740629685166+0j)
(0.309407796101949+0j)




What does this eigenvector represent though? Eigenvectors of a stochastic matrix
represent state state probabilities. This means that the long term probabilities
of being in each of the given nodes is given by the above vector.<br>

I don't completely understand the implications behind one of the values in the
abovematrix being less than 0. Frankly it makes me think something has gone
wrong, but for now this is what we have!

We can also use power iteration to determine the eigenvector for this matrix.

In [77]:
aa = (np.linalg.matrix_power(a,200))
cc = (np.linalg.matrix_power(c,200))
print("**aa:\n", aa[:,0])
print("**cc:\n", cc[:,0])

**aa:
 [0.16925 0.18054 0.19859 0.22026 0.23136]
**cc:
 [0.11713 0.14055 0.18272 0.25019 0.30941]


## Problem 4

First we need to develop a transition matrix for this. And for that, we need the
adjacency matrix

In [9]:
B = np.array([[0, 1, 1, 1, 1],
              [1, 0, 0, 0, 0],
              [1, 1, 0, 0, 0],
              [1, 0, 1, 0, 0],
              [0, 1, 0, 1, 0]])
big_b = make_transition(B, .8)
print(big_b)

[[0.1600 0.2267 0.2600 0.2600 0.3600]
 [0.2267 0.1600 0.1600 0.1600 0.1600]
 [0.2267 0.2267 0.1600 0.1600 0.1600]
 [0.2267 0.1600 0.2600 0.1600 0.1600]
 [0.1600 0.2267 0.1600 0.2600 0.1600]]


### Analysis of the $p = .8$ case

In [10]:
e_vlb, e_vcb = np.linalg.eig(big_b)
print(e_vlb)


[ 1.        +0.j         -0.07788218+0.07338108j -0.07788218-0.07338108j
 -0.02211782+0.05824321j -0.02211782-0.05824321j]


So the eigenvalue is at index 1

In [11]:
e = e_vcb[:,0]
print(e)

e_n = normalize(e)
norm_inf(e_n)

[0.55116102+0.j 0.39175186+0.j 0.41786866+0.j 0.43353873+0.j
 0.42447846+0.j]
NORMALISED VECTOR:
 [0.24840514+0.j 0.17656034+0.j 0.18833103+0.j 0.19539345+0.j
 0.19131003+0.j] 

NORMALISATION TEST:
(0.9999999999999999+0j)

FORMATTED VECTOR:
(0.248405144227276+0j)
(0.17656034294848497+0j)
(0.18833103247838412+0j)
(0.1953934461963234+0j)
(0.19131003414953138+0j)




### Analysis of $p=.4$

In [12]:
big_b2 = make_transition(B, 0.4)
e_vlb2, e_vcb2 = np.linalg.eig(big_b2)
print(e_vlb2)


[ 1.        +0.j         -0.23364655+0.22014324j -0.23364655-0.22014324j
 -0.06635345+0.17472963j -0.06635345-0.17472963j]


In [13]:
e2 = e_vcb2[:,0]
print(e)

e_n2 = normalize(e2)
norm_inf(e_n2)

[0.55116102+0.j 0.39175186+0.j 0.41786866+0.j 0.43353873+0.j
 0.42447846+0.j]
NORMALISED VECTOR:
 [0.31989529-0.j 0.14397906-0.j 0.17277487-0.j 0.19581152-0.j
 0.16753927-0.j] 

NORMALISATION TEST:
(1+0j)

FORMATTED VECTOR:
(0.31989528795811517-0j)
(0.14397905759162308-0j)
(0.17277486910994766-0j)
(0.1958115183246073-0j)
(0.16753926701570687-0j)




In [14]:
import numpy as np
import json
float_formatter = "{:.5f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
def bv(it,items):
    return np.array([1.0 if i == items.index(it)
                     else 0.0 for i in range(len(items))])
## >>> bv("c",["a","b","c","d"])
## array([0.00000, 0.00000, 1.00000, 0.00000])
def adj_from_json(json_file):
    with open(json_file) as f:
        adj_data = json.load(f)
    dict = {}
    
    for i in adj_data:
        lfrom = i['from']
        lto   = i['to']
        if lfrom in dict.keys():
            dict[lfrom].add(lto)
        else:
            dict[lfrom] = set()
            dict[lfrom].add(lto)
        if not(lto in dict.keys()):
            dict[lto] = set()
            
    sites = list(dict.keys())
    
    A = np.array([sum([bv(l_to, sites) for l_to in dict[l_from]],
                      np.zeros(len(sites)))
                  for l_from in sites]) 
    return (sites, A)
    

In [17]:
(ll, A) = adj_from_json("data.json")
print(ll)

['Blue Whale', 'Snail', 'Alligator', 'Lark', 'Fruit Bat', 'Gerbil', 'Hyena', 'Starfish', 'Carp', 'Bee', 'Giraffe', 'Asp', 'Domestic Canary', 'Buffalo', 'Star-Nosed Mole', 'Arctic Wolf', 'Porcupine', 'Reindeer', 'Muskox', 'Leopard', 'Armadillo', 'Crane Fly', 'Emu', 'Badger', 'Takin', 'Heron', 'Basilisk', 'Penguin', 'Beetle', 'Amphibian', 'Rhinoceros', 'Bonobo', 'Jellyfish', 'Fowl', 'Donkey', 'Arabian Leopard', 'Lemur', 'Swift', 'Aphid', 'Locust', 'Peafowl', 'Whale', 'Bald Eagle', 'Woodpecker', 'Minnow', 'Alpaca', 'Raven', 'Crocodile', 'Bat', 'Jay', 'Albatross', 'Possum', 'Ape', 'Mosquito', 'Robin', 'Vicuna', 'Elephant', 'Harrier', 'Falcon', 'Mite', 'Swan', 'Anglerfish', 'Anteater', 'Arrow Crab', 'Arctic Fox', 'Sheep', 'Ant', 'Ground Shark', 'Turtle', 'Aardvark', 'Squirrel', 'Bison', 'Meadowlark', 'Ocelot', 'Ground Sloth', 'Baboon', 'Anaconda', 'Grouse', 'Aardwolf', 'Guan', 'Panda', 'Dung Beetle', 'Scale Insect', 'Reptile', 'Antlion', 'Antelope', 'Giant Squid', 'Rook', 'Dingo', 'Salmon',

In [21]:
A_a = make_transition(A, 0.8)
e_vlA, e_vcA = np.linalg.eig(A_a)
print(e_vlA)

[ 1.00000000e+00+0.j          5.81335370e-02+0.09862851j
  5.81335370e-02-0.09862851j  1.10155151e-01+0.j
  8.09646903e-02+0.07090688j  8.09646903e-02-0.07090688j
  9.32008734e-02+0.04660742j  9.32008734e-02-0.04660742j
 -1.56819457e-02+0.10588806j -1.56819457e-02-0.10588806j
  2.89649299e-02+0.09979826j  2.89649299e-02-0.09979826j
  9.93204040e-02+0.01613876j  9.93204040e-02-0.01613876j
  9.42368093e-02+0.00672646j  9.42368093e-02-0.00672646j
  6.89643622e-03+0.1030294j   6.89643622e-03-0.1030294j
  4.86905798e-02+0.0823792j   4.86905798e-02-0.0823792j
  8.21566725e-02+0.03741955j  8.21566725e-02-0.03741955j
 -1.14948480e-01+0.j         -1.00611645e-01+0.04570818j
 -1.00611645e-01-0.04570818j -7.94183273e-02+0.07085312j
 -7.94183273e-02-0.07085312j -1.06389167e-01+0.j
 -1.01089822e-01+0.03183306j -1.01089822e-01-0.03183306j
 -1.31544158e-02+0.09723973j -1.31544158e-02-0.09723973j
 -7.54377898e-02+0.06346874j -7.54377898e-02-0.06346874j
  4.38950297e-03+0.08976249j  4.38950297e-03-0.08

In [83]:
eA = e_vcA[:,0]
e_nA = normalize(eA)
norm_inf(e_nA)

NORMALISED VECTOR:
 [0.01353617-0.j 0.01061748-0.j 0.01035092-0.j 0.01104113-0.j
 0.01052326-0.j 0.01188469-0.j 0.01002245-0.j 0.01031449-0.j
 0.01182667-0.j 0.01021592-0.j 0.01012033-0.j 0.01006833-0.j
 0.00988444-0.j 0.01014988-0.j 0.01010244-0.j 0.01080226-0.j
 0.01158824-0.j 0.0101065 -0.j 0.01061071-0.j 0.01039863-0.j
 0.01034989-0.j 0.01061232-0.j 0.01026845-0.j 0.01069608-0.j
 0.00983503-0.j 0.01155814-0.j 0.01082512-0.j 0.0114764 -0.j
 0.01043247-0.j 0.01071333-0.j 0.01045341-0.j 0.01052212-0.j
 0.00996873-0.j 0.01190911-0.j 0.01271324-0.j 0.01016368-0.j
 0.01172406-0.j 0.00999422-0.j 0.01045753-0.j 0.01069652-0.j
 0.01161354-0.j 0.01092861-0.j 0.00985734-0.j 0.01069072-0.j
 0.01131682-0.j 0.01045158-0.j 0.01028739-0.j 0.01021389-0.j
 0.01144867-0.j 0.01062477-0.j 0.01181791-0.j 0.01062761-0.j
 0.01082268-0.j 0.01098041-0.j 0.01075465-0.j 0.01066789-0.j
 0.01063725-0.j 0.01159582-0.j 0.01021821-0.j 0.01061801-0.j
 0.01058249-0.j 0.01053079-0.j 0.01009816-0.j 0.00995407-0.j
 0.0

In [84]:
def top_ten(e_nA):
    ## copy array to temp array so we don't accidentally change values in eigenvector
    temp = []
    for i in range(len(e_nA)):
        temp.append(e_nA[i])

    ## array of max values
    maxes = []

    ## loop through 10 times finding max val in array, and copying
    for i in range(10):

        ## find max val and index of max val
        max = np.amax(temp)
        ind = np.where(temp == max)[0]

        ## find the associated animal in ll
        animal = ll[int(ind)]

        ## set index in temp to zero so that it doesn't find the same max twice
        temp[int(ind)] = 0
        
        ## put the animal in the aray maxes
        maxes.append(animal)
        
    ## print the array out nicely
    print(maxes)
    for i in range(len(maxes)):
        print(maxes[i])

        
top_ten(e_nA)

['Blue Whale', 'Ant', 'Donkey', 'Squirrel', 'Rook', 'Grouse', 'Fowl', 'Gerbil', 'Carp', 'Albatross']
Blue Whale
Ant
Donkey
Squirrel
Rook
Grouse
Fowl
Gerbil
Carp
Albatross


Now we try and find the same thing but with the power iteration method.

In [82]:
A_aa = (np.linalg.matrix_power(A_a, 200))
top_ten(A_aa[:,0])

Blue Whale
Ant
Donkey
Squirrel
Rook
Grouse
Fowl
Gerbil
Carp
Albatross
